# Testing a new approach in ionBench
In this notebook, we will explore how you can take a python implementation of an optimiser and use it in ionBench. 


In [1]:
import copy
import numpy as np
import ionbench

# Define the optimiser
We will use a simple optimiser that performs a basic gradient descent algorithm.

In [2]:
def gradient_descent(f, df, initial_guess, lr=0.01, tol=1e-6, max_iter=100):
    x = copy.deepcopy(initial_guess)  # Initial guess - Think carefully about whether you want to edit initial_guess in place
    print(f"Starting parameters: {x}")
    print(f"Starting cost: {f(x)}")
    for i in range(max_iter):
        grad = df(x)  # Find the local gradient
        x -= lr * grad  # Take a step in the direction of the -gradient 
        print(f"Iteration {i}")
        print(f"Parameters: {x}")
        print(f"Cost: {f(x)}")
        if np.linalg.norm(grad) < tol:
            # If locally flat, then terminate
            break
    return x

# Testing the optimiser
First, we'll check our optimiser is working before we try to use it in ionBench.
We will test it against a simple quadratic function, trying to find the minimum at `x=1`.

In [3]:
x0 = 0
f = lambda x: (x - 1)**2
df = lambda x: 2 * (x - 1)
x_opt = gradient_descent(f, df, x0)
print(f"Optimiser parameter is {x_opt}")

Starting parameters: 0
Starting cost: 1
Iteration 0
Parameters: 0.02
Cost: 0.9603999999999999
Iteration 1
Parameters: 0.039599999999999996
Cost: 0.9223681600000001
Iteration 2
Parameters: 0.058808
Cost: 0.885842380864
Iteration 3
Parameters: 0.07763184000000001
Cost: 0.8507630225817856
Iteration 4
Parameters: 0.0960792032
Cost: 0.8170728068875469
Iteration 5
Parameters: 0.114157619136
Cost: 0.7847167237348001
Iteration 6
Parameters: 0.13187446675328002
Cost: 0.7536419414749018
Iteration 7
Parameters: 0.14923697741821443
Cost: 0.7237977205924958
Iteration 8
Parameters: 0.16625223786985013
Cost: 0.695135330857033
Iteration 9
Parameters: 0.18292719311245315
Cost: 0.6676079717550943
Iteration 10
Parameters: 0.19926864925020407
Cost: 0.6411706960735928
Iteration 11
Parameters: 0.2152832762652
Cost: 0.6157803365090785
Iteration 12
Parameters: 0.230977610739896
Cost: 0.5913954351833189
Iteration 13
Parameters: 0.2463580585250981
Cost: 0.5679761759500596
Iteration 14
Parameters: 0.261430897354

# Using the optimiser in ionBench
Now that we have seen the optimiser working, we will look at how this code needs to change to work with ionBench. 

Firstly, we need to change the function signature to take an ionBench Benchmarker object, rather than the function and its gradient.

Next, we need to change any cost and gradient calls to use this new Benchmarker object.

Finally, we should rename `initial_guess` to `x0` to match the ionBench API (this is used by `ionBench.multistart`).

In [4]:
def ionbench_gradient_descent(bm, x0, lr=1e-4, tol=1e-6, max_iter=100):
    x = copy.deepcopy(x0)  # Initial guess
    print(f"Starting parameters: {x}")
    print(f"Starting cost: {bm.cost(x)}")
    for i in range(max_iter):
        grad = bm.grad(x)  # Find the local gradient
        x -= lr * grad  # Take a step in the direction of the -gradient 
        print(f"Iteration {i}")
        print(f"Parameters: {x}")
        print(f"Cost: {bm.cost(x)}")
        if np.linalg.norm(grad) < tol:
            # If locally flat, then terminate
            break
    return x

# Testing the optimiser in ionBench


In [5]:
bm = ionbench.problems.loewe2016.IKr(sensitivities=True)
bm.plotter=False
x0 = bm.sample()
x_opt = ionbench_gradient_descent(bm, x0)
bm.evaluate()

Initialising Loewe 2016 IKr benchmark
Benchmarker initialised
Starting parameters: [ 3.96983773e-05  1.79937089e+01  7.21619706e+00 -3.36333427e+01
  5.72783381e-01  2.45777886e-01  1.56151240e+01  3.82248546e+00
  5.50466596e+01  3.37774528e+00  2.66266373e-01  6.33325210e+02]
Starting cost: 0.5485281798573118
Iteration 0
Parameters: [-7.58040982e-01  1.79937082e+01  7.21619691e+00 -3.36333424e+01
  5.72783381e-01  2.45764600e-01  1.56151240e+01  3.82248553e+00
  5.50466596e+01  3.37774529e+00  2.66124510e-01  6.33325210e+02]
Cost: 1.6640006326750434
Iteration 1
Parameters: [-7.58039748e-01  1.79937081e+01  7.21619486e+00 -3.36333418e+01
  5.72783381e-01  2.45943437e-01  1.56151229e+01  3.82248674e+00
  5.50466596e+01  3.37774534e+00  2.65545051e-01  6.33325210e+02]
Cost: 1.6603238379687733
Iteration 2
Parameters: [-7.58038517e-01  1.79937080e+01  7.21619282e+00 -3.36333412e+01
  5.72783381e-01  2.46121794e-01  1.56151218e+01  3.82248794e+00
  5.50466596e+01  3.37774538e+00  2.6496573

Our optimiser appears to be working, although not very well. 

Note the 101 cost function calls that don't do anything in the optimiser.

The first thing we will change is to remove the print statements, as these are not needed in ionBench.
In particular, finding the cost every iteration is very expensive and doesn't affect the optimisation.

Note: ionBench will try to be smart about some model evaluations. It will try to ignore any duplicate model evaluations, so if you call `bm.cost(x)` twice in a row, it will treat it as only one call in the results. It will also try to ignore any calls to `bm.cost(x)` if `bm.grad(x)` was called at the same parameters. Unfortunately, the calls to `bm.cost(x)` and `bm.grad(x)` are different (or at least the call to `bm.cost(x)` comes before the call to `bm.grad(x)`).

In [6]:
def ionbench_gradient_descent(bm, x0, lr=1e-4, tol=1e-6, max_iter=100):
    x = copy.deepcopy(x0)  # Initial guess
    for i in range(max_iter):
        grad = bm.grad(x)  # Find the local gradient
        x -= lr * grad  # Take a step in the direction of the -gradient 
        if np.linalg.norm(grad) < tol:
            # If locally flat, then terminate
            break
    return x

bm.reset()
ionbench_gradient_descent(bm, x0)
bm.evaluate()


===      Evaluating Performance      ===

Number of cost evaluations:      0
Number of grad evaluations:      100
Best cost:                       0.548538
Convergence reason:              Unknown optimiser specific termination.
Cost evaluations at convergence: 0
Grad evaluations at convergence: 100
Best cost at convergence:        0.548538
Model solve time at convergence: 0.000000
Grad solve time at convergence:  25.472000


# Matching the ionBench style
While the above optimiser will work with ionBench, there are a few style choices in the other ionBench optimisers that would be good to match.

* The initial guess should be optional, defaulting to None which will implement `bm.sample()`,
* The optimiser should call `bm.evaluate()` before returning,
* Terminating if the approach has converged (as defined by ionBench),
* The optimiser should set the max iteration flag if this occurred during the optimisation.

In [7]:
def ionbench_gradient_descent(bm, x0=None, lr=1e-4, tol=1e-6, max_iter=100):
    if x0 is None:
        x0 = bm.sample()
    x = copy.deepcopy(x0)  # Initial guess
    for i in range(max_iter):
        grad = bm.grad(x)  # Find the local gradient
        x -= lr * grad  # Take a step in the direction of the -gradient 
        if np.linalg.norm(grad) < tol or bm.is_converged():
            break
    if i >= max_iter-1:
        bm.set_max_iter_flag()
    bm.evaluate()
    return x

bm.reset()
ionbench_gradient_descent(bm, x0)
bm.evaluate()


===      Evaluating Performance      ===

Number of cost evaluations:      0
Number of grad evaluations:      100
Best cost:                       0.548538
Convergence reason:              Maximum iterations reached.
Cost evaluations at convergence: 0
Grad evaluations at convergence: 100
Best cost at convergence:        0.548538
Model solve time at convergence: 0.000000
Grad solve time at convergence:  23.300000


===      Evaluating Performance      ===

Number of cost evaluations:      0
Number of grad evaluations:      100
Best cost:                       0.548538
Convergence reason:              Maximum iterations reached.
Cost evaluations at convergence: 0
Grad evaluations at convergence: 100
Best cost at convergence:        0.548538
Model solve time at convergence: 0.000000
Grad solve time at convergence:  23.300000


Now we see that ionBench reports that the optimisation terminated due to the maximum number of iterations being reached.

# Making an approach
Now that we have a working optimiser, we can pair it with a modification to create our approach.
For our modification, we will use:
* Log transforms,
* Rate bounds,
* and Parameter bounds.

In [8]:
bm.reset() # Remember bm.reset() also resets modifications, if you don't want this then do bm.reset(fullReset=False)
mod = ionbench.modification.Modification(name='new modification', logTransform='on', parameterBounds='on', rateBounds='on', scaleFactors='off')
mod.apply(bm)
# Apply the transform to x0 (wouldn't be necessary if we did a new `x0=bm.sample()` call here)
transformedx0 = bm.input_parameter_space(x0)
ionbench_gradient_descent(bm, transformedx0);


===      Evaluating Performance      ===

Number of cost evaluations:      0
Number of grad evaluations:      100
Best cost:                       0.542235
Convergence reason:              Maximum iterations reached.
Cost evaluations at convergence: 0
Grad evaluations at convergence: 100
Best cost at convergence:        0.542235
Model solve time at convergence: 0.000000
Grad solve time at convergence:  16.099000


This has improved the convergence of the optimiser (the best cost is lower). Now we can look at testing this approach with `ionBench.multistart` against the same parameters as the original paper.

In [9]:
np.random.seed(0)  # The set seed for the initial run of ionBench
bm.reset(fullReset=False) # Keep the modification
ionbench.utils.multistart(opt=ionbench_gradient_descent, bm=bm, initParams=bm.sample(10), filename='');


===      Evaluating Performance      ===

Number of cost evaluations:      0
Number of grad evaluations:      100
Best cost:                       0.259441
Convergence reason:              Maximum iterations reached.
Cost evaluations at convergence: 0
Grad evaluations at convergence: 100
Best cost at convergence:        0.259441
Model solve time at convergence: 0.000000
Grad solve time at convergence:  25.990000

[-7.88693775 39.92272363  2.08267479  8.71878688  1.2822942   0.67246299
  6.61043947  3.67481362 70.63954153  2.57267894 -2.18475624  5.06939527]

===      Evaluating Performance      ===

Number of cost evaluations:      0
Number of grad evaluations:      100
Best cost:                       0.314284
Convergence reason:              Maximum iterations reached.
Cost evaluations at convergence: 0
Grad evaluations at convergence: 100
Best cost at convergence:        0.314284
Model solve time at convergence: 0.000000
Grad solve time at convergence:  25.947000

[ -7.79844262  65

# Next steps
Well, our approach didn't do very well. It failed to find the minimum/succeed in any of the runs (since the "Convergence reason" was always "Maximum iterations reached" and never "Cost threshold"). 

If we wanted to carry on evaluating this approach, we could:
* Run for longer until we observed some successes
* Evaluate the ERT of the approach using `ionbench.utils.results` (consider checking out the script `reviewOptimiserRuns.py` which does this for all the approaches using outputs from multistart, which can be saved by supplying a filename to multistart)
* Evaluate the significance, which can be done with the script `significance.py` (although this requires a lot of runs to be meaningful) which operates of the csv output of `reviewOptimiserRuns.py`